In [1]:
# feed the feature engineered one hot encoded but unscaled data (can try scaled too) to boosted trees. Then keep a label encoded feature engineered copy
# to predict using the models logistic regression and boosted trees. Also try the bagging ensemble of logistic regression.

### Pre Processing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.getcwd()

'/content'

In [4]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/Data files/Special data files/train.csv")
test =  pd.read_csv("/content/drive/MyDrive/Data files/Special data files/test.csv")

In [5]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
y = train["Survived"]
train = train.drop(columns = "Survived")
train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
data = pd.concat([train,test])
data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [9]:
data["Family"] = data["SibSp"] + data["Parch"]
data.drop(columns = ["SibSp","Parch"],inplace = True)
data

,PassengerId,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
0,1,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,PC 17758,108.9000,C105,C,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,359309,8.0500,NaN,S,0


In [10]:
data.drop(columns = "Cabin",inplace = True)
data

,PassengerId,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Family
0,1,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,A.5. 3236,8.0500,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,PC 17758,108.9000,C,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,SOTON/O.Q. 3101262,7.2500,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,359309,8.0500,S,0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   Ticket       1309 non-null   object 
 6   Fare         1308 non-null   float64
 7   Embarked     1307 non-null   object 
 8   Family       1309 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.3+ KB


In [12]:
data.nunique()

,0
PassengerId,1309
Pclass,3
Name,1307
Sex,2
Age,98
Ticket,929
Fare,281
Embarked,3
Family,9


In [13]:
# encode categroicals - extract title from names, one hot encode sex, label encode ticket because there are 929 unique values cant one hot,
# one hot encode embarked
# impute age, fare and embarked

In [14]:
# group data by survied and fare amount
# data.groupby("Survived")["Fair"]

In [15]:
data = pd.get_dummies(data,prefix = "Embarked", columns = ["Embarked"], dtype = int)
data

,PassengerId,Pclass,Name,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,1,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,1,1,0,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,1,0,0,1
4,5,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,A.5. 3236,8.0500,0,0,0,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,PC 17758,108.9000,0,1,0,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,SOTON/O.Q. 3101262,7.2500,0,0,0,1
416,1308,3,"Ware, Mr. Frederick",male,NaN,359309,8.0500,0,0,0,1


In [16]:
names = data["Name"]
data.drop(columns = "Name",inplace = True)
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,1,3,male,22.0,A/5 21171,7.2500,1,0,0,1
1,2,1,female,38.0,PC 17599,71.2833,1,1,0,0
2,3,3,female,26.0,STON/O2. 3101282,7.9250,0,0,0,1
3,4,1,female,35.0,113803,53.1000,1,0,0,1
4,5,3,male,35.0,373450,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,A.5. 3236,8.0500,0,0,0,1
414,1306,1,female,39.0,PC 17758,108.9000,0,1,0,0
415,1307,3,male,38.5,SOTON/O.Q. 3101262,7.2500,0,0,0,1
416,1308,3,male,NaN,359309,8.0500,0,0,0,1


In [17]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["Sex"] = encoder.fit_transform(data.Sex)
data["Ticket"] = encoder.fit_transform(data.Ticket)
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,1,3,1,22.0,720,7.2500,1,0,0,1
1,2,1,0,38.0,816,71.2833,1,1,0,0
2,3,3,0,26.0,914,7.9250,0,0,0,1
3,4,1,0,35.0,65,53.1000,1,0,0,1
4,5,3,1,35.0,649,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,NaN,711,8.0500,0,0,0,1
414,1306,1,0,39.0,834,108.9000,0,1,0,0
415,1307,3,1,38.5,872,7.2500,0,0,0,1
416,1308,3,1,NaN,579,8.0500,0,0,0,1


In [18]:
data.nunique()

,0
PassengerId,1309
Pclass,3
Sex,2
Age,98
Ticket,929
Fare,281
Family,9
Embarked_C,2
Embarked_Q,2
Embarked_S,2


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   int64  
 3   Age          1046 non-null   float64
 4   Ticket       1309 non-null   int64  
 5   Fare         1308 non-null   float64
 6   Family       1309 non-null   int64  
 7   Embarked_C   1309 non-null   int64  
 8   Embarked_Q   1309 non-null   int64  
 9   Embarked_S   1309 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 112.5 KB


In [20]:
# Embarked got encoded wihtout imputing 2 value :( , ok by now all features are numerical so we need to do imputing of age and fare

In [21]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Imputer = IterativeImputer()
data["Age"] = Imputer.fit_transform(data[["Age"]]) # double square brackets to convert 2d array to 1d array used by the imputer
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,1,3,1,22.000000,720,7.2500,1,0,0,1
1,2,1,0,38.000000,816,71.2833,1,1,0,0
2,3,3,0,26.000000,914,7.9250,0,0,0,1
3,4,1,0,35.000000,65,53.1000,1,0,0,1
4,5,3,1,35.000000,649,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1
414,1306,1,0,39.000000,834,108.9000,0,1,0,0
415,1307,3,1,38.500000,872,7.2500,0,0,0,1
416,1308,3,1,29.881138,579,8.0500,0,0,0,1


In [22]:
data["Fare"] = Imputer.fit_transform(data[["Fare"]])

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   int64  
 3   Age          1309 non-null   float64
 4   Ticket       1309 non-null   int64  
 5   Fare         1309 non-null   float64
 6   Family       1309 non-null   int64  
 7   Embarked_C   1309 non-null   int64  
 8   Embarked_Q   1309 non-null   int64  
 9   Embarked_S   1309 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 112.5 KB


In [24]:
# ok so preprocessing done, but i wanted to group survived by fare prices to maybe create a one hot feature like Fare_above_100 and such but
# cant do so because i combined train test data for efficient and similar preprocessing across both train and test sets
# Also i wanted to extract titles from names but dont know the code snippet now and im offline as well, but i had saved the feature as "Names"

In [25]:
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,1,3,1,22.000000,720,7.2500,1,0,0,1
1,2,1,0,38.000000,816,71.2833,1,1,0,0
2,3,3,0,26.000000,914,7.9250,0,0,0,1
3,4,1,0,35.000000,65,53.1000,1,0,0,1
4,5,3,1,35.000000,649,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1
414,1306,1,0,39.000000,834,108.9000,0,1,0,0
415,1307,3,1,38.500000,872,7.2500,0,0,0,1
416,1308,3,1,29.881138,579,8.0500,0,0,0,1


In [26]:
x = data.iloc[:891].copy()
leaderboard_test = data.iloc[891:].copy()
x.drop(columns = "PassengerId",inplace = True)
Test_ID = leaderboard_test["PassengerId"]
leaderboard_test.drop(columns = "PassengerId",inplace = True)
x
leaderboard_test
y
Test_ID

,PassengerId
0,892
1,893
2,894
3,895
4,896
...,...
413,1305
414,1306
415,1307
416,1308


In [27]:
# I seperated the train set (x) with train target variable y , and leaderbaord test set (named it so because during creating train test split the
# name "test" would clash)

In [28]:
x

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.000000,720,7.2500,1,0,0,1
1,1,0,38.000000,816,71.2833,1,1,0,0
2,3,0,26.000000,914,7.9250,0,0,0,1
3,1,0,35.000000,65,53.1000,1,0,0,1
4,3,1,35.000000,649,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,1,27.000000,130,13.0000,0,0,0,1
887,1,0,19.000000,18,30.0000,0,0,0,1
888,3,0,29.881138,923,23.4500,3,0,0,1
889,1,1,26.000000,11,30.0000,0,1,0,0


In [29]:
# I'll run logistic regression using train test split and cross val score on this simple preprocessed non feature engineered data to establish a
# base model performance, i'll also use my own numpy coded logistic regression model on it!

In [30]:
# now to do : scale the data x, add regularizer, then predict on train-test split and leaderboard test using my model from scratch

### Boosted trees on simple preprocessed data

In [38]:
# unrelated cell to the section, but i just wanted to directly use the boosted trees on unscaled preprocessed data for logging accuracy
xgb_train,xgb_test,ygb_train,ygb_test = splitter(x,y,test_size = 0.3)


In [39]:
# first i'll use xgboost
from xgboost import XGBClassifier
mz = XGBClassifier()
mz.fit(xgb_train,ygb_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [40]:
pz = mz.predict(xgb_test)
from sklearn.metrics import accuracy_score as accuracy
accuracy(ygb_test,pz) #82.835

0.8283582089552238

In [43]:
accuracy(ygb_train,mz.predict(xgb_train))   #wtff, really? 99.83

0.9983948635634029

In [44]:
# lets predict on leaderboard test on spot
py = mz.predict(leaderboard_test)
res_xgb = pd.DataFrame({"PassengerId":Test_ID,"Survived":py})
res_xgb.to_csv("Xgb_preprocessed.csv",index=False,header=True)

In [46]:
from lightgbm import LGBMClassifier
my_light = LGBMClassifier()
my_light.fit(xgb_train,ygb_train)
plight = my_light.predict(xgb_test)
accuracy(ygb_test,plight) #82.46

[LightGBM] [Info] Number of positive: 232, number of negative: 391
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 398
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.372392 -> initscore=-0.521970
[LightGBM] [Info] Start training from score -0.521970
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


0.8246268656716418

In [47]:
accuracy(ygb_train,my_light.predict(xgb_train)) #these boosted trees really got overfit, 98.71%

0.9871589085072231

In [52]:
py_light = my_light.predict(leaderboard_test)
res_lgb = pd.DataFrame({"PassengerId":Test_ID,"Survived":py_light})
res_lgb.to_csv("Lgb_preprocessed_true.csv",index=False,header=True) #leaderboard accuracy 76.315

In [50]:
!pip install catboost
from catboost import CatBoostClassifier
m_cat = CatBoostClassifier()
m_cat.fit(xgb_train,ygb_train)
accuracy(ygb_test,m_cat.predict(xgb_test)) #83.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.4 MB/s eta 0:00:00
Learning rate set to 0.008417
0:	learn: 0.6868706	total: 49.3ms	remaining: 49.2s
1:	learn: 0.6815550	total: 53.1ms	remaining: 26.5s
2:	learn: 0.6754567	total: 54.7ms	remaining: 18.2s
3:	learn: 0.6703751	total: 56.2ms	remaining: 14s
4:	learn: 0.6653260	total: 62ms	remaining: 12.3s
5:	learn: 0.6601888	total: 66.5ms	remaining: 11s
6:	learn: 0.6548819	total: 68.1ms	remaining: 9.66s
7:	learn: 0.6500169	total: 69.7ms	remaining: 8.64s
8:	learn: 0.6445429	total: 75.4ms	remaining: 8.3s
9:	learn: 0.6396054	total: 76.9ms	remaining: 7.62s
10:	learn: 0.6349140	total: 78.5ms	remaining: 7.06s
11:	learn: 0.6300574	total: 85ms	remaining: 7s
12:	learn: 0.6254749	total: 86.6ms	remaining: 6.57s
13:	learn: 0.6211395	total: 87.4ms	remaining: 6.16s
14:	learn: 0.6161414	total: 92.1ms	remaining: 6.05s
15:	learn: 0.6114866	total: 96.8ms	remaining: 5.95s
16:	learn: 0.6073132	total: 98.2ms	remaining: 5.68s
17:	learn: 0.6027733	total: 10

0.832089552238806

In [51]:
accuracy(ygb_train,m_cat.predict(xgb_train)) #93.2584

0.9325842696629213

In [53]:
py_cat = m_cat.predict(leaderboard_test)
res_catgb = pd.DataFrame({"PassengerId":Test_ID,"Survived":py_cat})
res_catgb.to_csv("catgb_preprocessed.csv",index=False,header=True) #leaderboard accuracy 76.076

In [ ]:
# conclusion , these trees seem to be overfitted to train set or there is data leakage, ohh god

### Logistic Regression From Scratch

In [31]:
# coding my logistic regression in numpy
y_col = y.values.reshape(-1,1) #messed up in dimensions so improving it

import numpy as np

def sigmoid(w,x,b):
    z = x@w + b
    g = 1/(1+np.exp(-z))
    return g

def compute_cost(w,x,b,y,lam):
    m=x.shape[0]
    pred=sigmoid(w,x,b)
    cost= (-((y.T @ np.log(pred)) + ((1-y).T @ np.log(1-pred)))/m) + (lam/(2*m))*np.sum(w**2)
    return cost.item()

def compute_gradient(w,x,b,y,lam):
  m = x.shape[0]
  pred = sigmoid(w,x,b)

  dj_dw = ((1/m) * (x.T @ (pred-y))) + (lam*w)/m  # dj_dw = ((1/m) * np.dot(x,(g.T - y))).sum(axis=0)
  dj_db = (1/m) * np.sum(pred - y)

  return dj_dw,dj_db

def run_gradient_descent(w,x,b,y,lam,alpha,iter):
  g = sigmoid(w,x,b)
  m = x.shape[0]

  for i in range(iter):
    cost = compute_cost(w,x,b,y,lam)

    dj_dw, dj_db = compute_gradient(w,x,b,y,lam)

    w-= alpha * dj_dw
    b-= alpha * dj_db

    if i%100 == 0 or i == iter-1:
      print(f"Iteration: {i}, cost: {cost:.3f}")

  return w,b

In [32]:
w = np.zeros((9,1)) #9 features in x
b = 0.0
x_np = x.to_numpy()
print(f"w shape: {w.shape}")
y_col.shape
# x_np.shape
w,b = run_gradient_descent(w,x_np,b,y_col,0.01, 0.000001, 2000)

w shape: (9, 1)
Iteration: 0, cost: 0.693
Iteration: 100, cost: 0.649
Iteration: 200, cost: 0.645
Iteration: 300, cost: 0.642
Iteration: 400, cost: 0.640
Iteration: 500, cost: 0.637
Iteration: 600, cost: 0.635
Iteration: 700, cost: 0.634
Iteration: 800, cost: 0.632
Iteration: 900, cost: 0.631
Iteration: 1000, cost: 0.629
Iteration: 1100, cost: 0.628
Iteration: 1200, cost: 0.627
Iteration: 1300, cost: 0.626
Iteration: 1400, cost: 0.625
Iteration: 1500, cost: 0.625
Iteration: 1600, cost: 0.624
Iteration: 1700, cost: 0.623
Iteration: 1800, cost: 0.623
Iteration: 1900, cost: 0.622
Iteration: 1999, cost: 0.622


In [33]:
w.T

array([[-1.97809325e-04, -2.42662136e-04, -1.29802045e-03,
        -1.42261328e-03,  7.27319464e-03,  3.12706950e-06,
         5.10544960e-05,  7.15658528e-06, -6.61919094e-05]])

In [34]:
b

np.float64(-6.024382775280305e-06)

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[ 0.82737724,  0.73769513, -0.59525389, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [-1.56610693, -1.35557354,  0.6359965 , ...,  2.0745051 ,
        -0.30756234, -1.61470971],
       [ 0.82737724, -1.35557354, -0.28744129, ..., -0.48204268,
        -0.30756234,  0.61930636],
       ...,
       [ 0.82737724, -1.35557354,  0.01122448, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [-1.56610693,  0.73769513, -0.28744129, ...,  2.0745051 ,
        -0.30756234, -1.61470971],
       [ 0.82737724,  0.73769513,  0.1742776 , ..., -0.48204268,
         3.25137334, -1.61470971]])

In [ ]:
# now i wanna split dataset into train test split and using my model then check accuracy

In [36]:
from sklearn.model_selection import train_test_split as splitter
x_train,x_test,y_train,y_test = splitter(x_scaled,y,test_size = 0.3)
x_train

array([[-1.56610693,  0.73769513, -0.82611333, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [-0.36936484, -1.35557354, -1.98041057, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [-0.36936484, -1.35557354,  0.3281839 , ..., -0.48204268,
        -0.30756234,  0.61930636],
       ...,
       [ 0.82737724,  0.73769513,  0.3281839 , ..., -0.48204268,
        -0.30756234,  0.61930636],
       [ 0.82737724,  0.73769513, -0.05658184, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [-0.36936484,  0.73769513, -2.05736372, ..., -0.48204268,
        -0.30756234,  0.61930636]])

In [ ]:
y_train.shape

(623,)

In [ ]:
def predict(w,x,b,threshold):
    pred = sigmoid(w,x,b)
    predict = (pred>=threshold).astype(int)
    return predict

In [ ]:
w = np.zeros((9,1))
b = 0.0
# x_train and x_test are 2d vectors but y_train and test are 1 d vectors so i need to convert them into 2d vectors
y_tr = y_train.values.reshape(-1,1)
y_te = y_test.values.reshape(-1,1)
y_tr.shape #(rerun this cell before running run_gradient_descent again to reset w,b)

(623, 1)

In [ ]:
w,b = run_gradient_descent(w,x_train,b,y_tr,0.01,0.0001,2000)

Iteration: 0, cost: 0.693
Iteration: 100, cost: 0.692
Iteration: 200, cost: 0.690
Iteration: 300, cost: 0.689
Iteration: 400, cost: 0.687
Iteration: 500, cost: 0.686
Iteration: 600, cost: 0.684
Iteration: 700, cost: 0.683
Iteration: 800, cost: 0.682
Iteration: 900, cost: 0.680
Iteration: 1000, cost: 0.679
Iteration: 1100, cost: 0.678
Iteration: 1200, cost: 0.676
Iteration: 1300, cost: 0.675
Iteration: 1400, cost: 0.673
Iteration: 1500, cost: 0.672
Iteration: 1600, cost: 0.671
Iteration: 1700, cost: 0.670
Iteration: 1800, cost: 0.668
Iteration: 1900, cost: 0.667
Iteration: 1999, cost: 0.666


In [ ]:
# cost doesnt seem to reduce much, means maybe lambda is too big or alpha is too low, ill manage both and increase iterations

In [ ]:
w,b = run_gradient_descent(w,x_train,b,y_tr,0.001,0.001,2500)

Iteration: 0, cost: 0.693
Iteration: 100, cost: 0.679
Iteration: 200, cost: 0.666
Iteration: 300, cost: 0.654
Iteration: 400, cost: 0.642
Iteration: 500, cost: 0.632
Iteration: 600, cost: 0.623
Iteration: 700, cost: 0.614
Iteration: 800, cost: 0.605
Iteration: 900, cost: 0.598
Iteration: 1000, cost: 0.591
Iteration: 1100, cost: 0.584
Iteration: 1200, cost: 0.578
Iteration: 1300, cost: 0.572
Iteration: 1400, cost: 0.566
Iteration: 1500, cost: 0.561
Iteration: 1600, cost: 0.556
Iteration: 1700, cost: 0.552
Iteration: 1800, cost: 0.547
Iteration: 1900, cost: 0.543
Iteration: 2000, cost: 0.539
Iteration: 2100, cost: 0.536
Iteration: 2200, cost: 0.532
Iteration: 2300, cost: 0.529
Iteration: 2400, cost: 0.526
Iteration: 2499, cost: 0.523


In [ ]:
# cost did decrease, i want to reduce it more though


In [ ]:
w,b = run_gradient_descent(w,x_train,b,y_tr,0.0001,0.01,3500)

Iteration: 0, cost: 0.693
Iteration: 100, cost: 0.590
Iteration: 200, cost: 0.539
Iteration: 300, cost: 0.510
Iteration: 400, cost: 0.491
Iteration: 500, cost: 0.479
Iteration: 600, cost: 0.470
Iteration: 700, cost: 0.464
Iteration: 800, cost: 0.459
Iteration: 900, cost: 0.456
Iteration: 1000, cost: 0.453
Iteration: 1100, cost: 0.451
Iteration: 1200, cost: 0.450
Iteration: 1300, cost: 0.448
Iteration: 1400, cost: 0.447
Iteration: 1500, cost: 0.446
Iteration: 1600, cost: 0.446
Iteration: 1700, cost: 0.445
Iteration: 1800, cost: 0.445
Iteration: 1900, cost: 0.444
Iteration: 2000, cost: 0.444
Iteration: 2100, cost: 0.443
Iteration: 2200, cost: 0.443
Iteration: 2300, cost: 0.443
Iteration: 2400, cost: 0.443
Iteration: 2500, cost: 0.443
Iteration: 2600, cost: 0.443
Iteration: 2700, cost: 0.442
Iteration: 2800, cost: 0.442
Iteration: 2900, cost: 0.442
Iteration: 3000, cost: 0.442
Iteration: 3100, cost: 0.442
Iteration: 3200, cost: 0.442
Iteration: 3300, cost: 0.442
Iteration: 3400, cost: 0.4

In [ ]:
# ok enough now i wanna predict on x_test, also it converges to 0.428 means it is the minimum cost

In [ ]:
pred = predict(w,x_test,b,0.5)

In [ ]:
pred

array([[0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
    

In [ ]:
# to check accuracy
from sklearn.metrics import accuracy_score as accuracy
accuracy(pred,y_te)

0.7985074626865671

In [ ]:
# damn! my model from scratch got 80.597% on cv!
# time to predict on leaderboard as well ;)
# lets first also predict on the train set to see training accuracy

In [ ]:
p2 = predict(w,x_train,b,0.5)
accuracy(p2,y_tr)  #scores changed when i reran the cells, f, to stablise this I'll need to use random state

0.8089887640449438

In [ ]:
# ohk nice, 79.293% on train set, seems like its underfit

In [ ]:
predictions = predict(w,leaderboard_test,b,0.5)


In [ ]:
predictions=(predictions.to_numpy()).reshape(418,)
predictions.shape

(418,)

In [ ]:
result1 = pd.DataFrame({"PassengerId":Test_ID,"Survived":predictions})
result1

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [ ]:
result1["Survived"].value_counts()

Survived
0    406
1     12
Name: count, dtype: int64

In [ ]:
predictions #wth i'll look at it later :(

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
result1.to_csv("Logistic Reg from scratch2.csv",header=True,index=False) #ok ill submit it though i know accuracy will be very less, there's some issue

![image.png](attachment:8ef502bb-18f4-449d-8243-1bde89937a8f.png)

In [ ]:
# it got 62.2% only, now i'll look for the issue

In [ ]:
# second attempt but stuck at 63.157, issue is most predictions are 0. considering there is no issue with my functions
# train accuracy = 83.306%
# cv accuracy =  77.985%
# leaderboard accuracy = 63.157%
# seems like model is overfit, but it should reflect in cv accuracy as well, so maybe my predict function is wrong
# ohhk now i see, i didnt scale the leader baord test set, yikes,lol

In [ ]:
scaled_leaderboard_test = scaler.transform(leaderboard_test)
p = predict(w,scaled_leaderboard_test,b,0.5)
p #now that looks sweet!

In [ ]:
p = p.reshape(418,)
result3 = pd.DataFrame({"PassengerId":Test_ID,"Survived":p})
result3["Survived"].value_counts()

Survived
0    264
1    154
Name: count, dtype: int64

In [ ]:
result3.to_csv("Logistic Reg from scratch3.csv",header=True,index=False)

![image.png](attachment:c31bb76e-e51e-4dcb-95f9-4affb7bd02f9.png)

In [ ]:
# It did work...yess!

In [ ]:
w.T

array([[-0.73407141, -1.33721458, -0.42440414, -0.22498804,  0.27944166,
        -0.35686605,  0.01487571,  0.00288802, -0.04944191]])

In [ ]:
b

-0.6780945346813919

In [ ]:
# so final, improved leaderboard score is 76.076%

In [ ]:
pn1 = predict(w,x_train,b,0.5)
print(accuracy(pn1,y_tr))
pn2 = predict(w,x_test,b,0.5)
accuracy(pn2,y_te)

0.8089887640449438


0.7798507462686567

In [ ]:
# train accuracy = 80.898%
# cv accuracy =  77.985%
# leaderboard accuracy = 76.076% , later i will also use my scratch model to predict after proper feature engineering and polynomial regression

### Feature Engineering

In [ ]:
data = pd.concat([x,leaderboard_test])
data

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.000000,720,7.2500,1,0,0,1
1,1,0,38.000000,816,71.2833,1,1,0,0
2,3,0,26.000000,914,7.9250,0,0,0,1
3,1,0,35.000000,65,53.1000,1,0,0,1
4,3,1,35.000000,649,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...
413,3,1,29.881138,711,8.0500,0,0,0,1
414,1,0,39.000000,834,108.9000,0,1,0,0
415,3,1,38.500000,872,7.2500,0,0,0,1
416,3,1,29.881138,579,8.0500,0,0,0,1


In [ ]:
names

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
413                                   Spector, Mr. Woolf
414                         Oliva y Ocana, Dona. Fermina
415                         Saether, Mr. Simon Sivertsen
416                                  Ware, Mr. Frederick
417                             Peter, Master. Michael J
Name: Name, Length: 1309, dtype: object

In [54]:
data["Names"] = names
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Names
0,1,3,1,22.000000,720,7.2500,1,0,0,1,"Braund, Mr. Owen Harris"
1,2,1,0,38.000000,816,71.2833,1,1,0,0,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,3,3,0,26.000000,914,7.9250,0,0,0,1,"Heikkinen, Miss. Laina"
3,4,1,0,35.000000,65,53.1000,1,0,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,5,3,1,35.000000,649,8.0500,0,0,0,1,"Allen, Mr. William Henry"
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1,"Spector, Mr. Woolf"
414,1306,1,0,39.000000,834,108.9000,0,1,0,0,"Oliva y Ocana, Dona. Fermina"
415,1307,3,1,38.500000,872,7.2500,0,0,0,1,"Saether, Mr. Simon Sivertsen"
416,1308,3,1,29.881138,579,8.0500,0,0,0,1,"Ware, Mr. Frederick"


In [55]:
data['Title'] = data["Names"].str.extract(r',\s*([^\.]+)\.', expand=False)

In [56]:
data = data.drop(columns = "Names")
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Title
0,1,3,1,22.000000,720,7.2500,1,0,0,1,Mr
1,2,1,0,38.000000,816,71.2833,1,1,0,0,Mrs
2,3,3,0,26.000000,914,7.9250,0,0,0,1,Miss
3,4,1,0,35.000000,65,53.1000,1,0,0,1,Mrs
4,5,3,1,35.000000,649,8.0500,0,0,0,1,Mr
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1,Mr
414,1306,1,0,39.000000,834,108.9000,0,1,0,0,Dona
415,1307,3,1,38.500000,872,7.2500,0,0,0,1,Mr
416,1308,3,1,29.881138,579,8.0500,0,0,0,1,Mr


In [ ]:
x["Survive"] = y
x #i wanted to know the relation of fare with survival

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Survive
0,3,1,22.000000,720,7.2500,1,0,0,1,0
1,1,0,38.000000,816,71.2833,1,1,0,0,1
2,3,0,26.000000,914,7.9250,0,0,0,1,1
3,1,0,35.000000,65,53.1000,1,0,0,1,1
4,3,1,35.000000,649,8.0500,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
886,2,1,27.000000,130,13.0000,0,0,0,1,0
887,1,0,19.000000,18,30.0000,0,0,0,1,1
888,3,0,29.881138,923,23.4500,3,0,0,1,0
889,1,1,26.000000,11,30.0000,0,1,0,0,1


In [ ]:
data.describe() #we see the distribution of fare is centered around pretty low values compared to the maximum, 75%ile at 35

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,2.294882,0.644003,29.881138,463.625668,33.295479,0.883881,0.206264,0.093965,0.698243
std,0.837836,0.478997,12.883193,278.069490,51.738879,1.583639,0.404777,0.291891,0.459196
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,212.000000,7.895800,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,29.881138,459.000000,14.454200,0.000000,0.000000,0.000000,1.000000
75%,3.000000,1.000000,35.000000,727.000000,31.275000,1.000000,0.000000,0.000000,1.000000
max,3.000000,1.000000,80.000000,928.000000,512.329200,10.000000,1.000000,1.000000,1.000000


In [ ]:
x[x["Fare"]>=14.4542]["Survive"].value_counts() #50 percent poeple had fare above 14, survival/death ratio is balanced around 1

,count
Survive,
1,231
0,220


In [ ]:
x[x["Fare"]>=50]["Survive"].value_counts() #we see nearly 2/3 people having fare above 50 survived, survivcal/death ratio of 2:1

,count
Survive,
1,109
0,52


In [ ]:
x[x["Fare"]>=60]["Survive"].value_counts() #here ratio is 2:1 again

,count
Survive,
1,82
0,40


In [ ]:
x[x["Fare"]>=100]["Survive"].value_counts() #here ratio is nearly 3:1 so fair seems to have good relation with survival

,count
Survive,
1,39
0,14


In [ ]:
x[x["Pclass"]==1]["Survive"].value_counts() #we see pclass also has strong relation with survival

,count
Survive,
1,136
0,80


In [ ]:
x[x["Pclass"]==2]["Survive"].value_counts()

Survive
0    97
1    87
Name: count, dtype: int64

In [ ]:
x[x["Pclass"]==3]["Survive"].value_counts()

Survive
0    372
1    119
Name: count, dtype: int64

In [ ]:
data

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Title
0,3,1,22.000000,720,7.2500,1,0,0,1,Mr
1,1,0,38.000000,816,71.2833,1,1,0,0,Mrs
2,3,0,26.000000,914,7.9250,0,0,0,1,Miss
3,1,0,35.000000,65,53.1000,1,0,0,1,Mrs
4,3,1,35.000000,649,8.0500,0,0,0,1,Mr
...,...,...,...,...,...,...,...,...,...,...
413,3,1,29.881138,711,8.0500,0,0,0,1,Mr
414,1,0,39.000000,834,108.9000,0,1,0,0,Dona
415,3,1,38.500000,872,7.2500,0,0,0,1,Mr
416,3,1,29.881138,579,8.0500,0,0,0,1,Mr


In [57]:
# so, i added Title feature, now ill add Fare bin and one hot encode it, ill also do something with pclas maybe...
data["FareBin"] = pd.cut(data["Fare"], bins=[0, 14.45, 50, 100, data["Fare"].max()], labels=["Low", "Mid", "High", "Very High"])
data #modified cell, excluded one hot encoding , ik this is getting messy , but yeah i've to do it for "Science"

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Title,FareBin
0,1,3,1,22.000000,720,7.2500,1,0,0,1,Mr,Low
1,2,1,0,38.000000,816,71.2833,1,1,0,0,Mrs,High
2,3,3,0,26.000000,914,7.9250,0,0,0,1,Miss,Low
3,4,1,0,35.000000,65,53.1000,1,0,0,1,Mrs,High
4,5,3,1,35.000000,649,8.0500,0,0,0,1,Mr,Low
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1,Mr,Low
414,1306,1,0,39.000000,834,108.9000,0,1,0,0,Dona,Very High
415,1307,3,1,38.500000,872,7.2500,0,0,0,1,Mr,Low
416,1308,3,1,29.881138,579,8.0500,0,0,0,1,Mr,Low


In [58]:
data["FareBin"] = encoder.fit_transform(data.FareBin) #new cell, labele encoded them all
data["Title"] = encoder.fit_transform(data.Title)
data

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Title,FareBin
0,1,3,1,22.000000,720,7.2500,1,0,0,1,12,1
1,2,1,0,38.000000,816,71.2833,1,1,0,0,13,0
2,3,3,0,26.000000,914,7.9250,0,0,0,1,9,1
3,4,1,0,35.000000,65,53.1000,1,0,0,1,13,0
4,5,3,1,35.000000,649,8.0500,0,0,0,1,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1,12,1
414,1306,1,0,39.000000,834,108.9000,0,1,0,0,3,3
415,1307,3,1,38.500000,872,7.2500,0,0,0,1,12,1
416,1308,3,1,29.881138,579,8.0500,0,0,0,1,12,1


In [ ]:
data = pd.get_dummies(data, columns=['Title']).astype(int)
data

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Bin_Low,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,3,1,22,720,7,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,1,0,38,816,71,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,0,26,914,7,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
3,1,0,35,65,53,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,3,1,35,649,8,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,1,29,711,8,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
414,1,0,39,834,108,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,3,1,38,872,7,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
416,3,1,29,579,8,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
data.info() #idk by mistake the fare bin column automitcally got converted to astype int maybe due to a deleted cell

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Pclass              1309 non-null   int64
 1   Sex                 1309 non-null   int64
 2   Age                 1309 non-null   int64
 3   Ticket              1309 non-null   int64
 4   Fare                1309 non-null   int64
 5   Family              1309 non-null   int64
 6   Embarked_C          1309 non-null   int64
 7   Embarked_Q          1309 non-null   int64
 8   Embarked_S          1309 non-null   int64
 9   Bin_Low             1309 non-null   int64
 10  Bin_Mid             1309 non-null   int64
 11  Bin_High            1309 non-null   int64
 12  Bin_Very High       1309 non-null   int64
 13  Title_Capt          1309 non-null   int64
 14  Title_Col           1309 non-null   int64
 15  Title_Don           1309 non-null   int64
 16  Title_Dona          1309 non-null   int64
 17  T

In [ ]:
data = pd.get_dummies(data, prefix="PClass", columns=['Pclass'], dtype = int)
data

,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Bin_Low,Bin_Mid,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,PClass_1,PClass_2,PClass_3
0,1,22,720,7,1,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
1,0,38,816,71,1,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0,26,914,7,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,35,65,53,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,1,35,649,8,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1,29,711,8,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
414,0,39,834,108,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
415,1,38,872,7,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
416,1,29,579,8,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Sex                 1309 non-null   int64
 1   Age                 1309 non-null   int64
 2   Ticket              1309 non-null   int64
 3   Fare                1309 non-null   int64
 4   Family              1309 non-null   int64
 5   Embarked_C          1309 non-null   int64
 6   Embarked_Q          1309 non-null   int64
 7   Embarked_S          1309 non-null   int64
 8   Bin_Low             1309 non-null   int64
 9   Bin_Mid             1309 non-null   int64
 10  Bin_High            1309 non-null   int64
 11  Bin_Very High       1309 non-null   int64
 12  Title_Capt          1309 non-null   int64
 13  Title_Col           1309 non-null   int64
 14  Title_Don           1309 non-null   int64
 15  Title_Dona          1309 non-null   int64
 16  Title_Dr            1309 non-null   int64
 17  T

In [ ]:
# feature engineering done on "data" variable but polynomial regression not done

### Sklearn Models

In [ ]:
# model from scratch
# train accuracy = 80.898%
# cv accuracy =  77.985%
# leaderboard accuracy = 76.076%

In [ ]:
x

,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.000000,720,7.2500,1,0,0,1
1,1,0,38.000000,816,71.2833,1,1,0,0
2,3,0,26.000000,914,7.9250,0,0,0,1
3,1,0,35.000000,65,53.1000,1,0,0,1
4,3,1,35.000000,649,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,1,27.000000,130,13.0000,0,0,0,1
887,1,0,19.000000,18,30.0000,0,0,0,1
888,3,0,29.881138,923,23.4500,3,0,0,1
889,1,1,26.000000,11,30.0000,0,1,0,0


In [ ]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
Test_ID

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [ ]:
# x_train, x_test, scaled_leaderboard test,, y_train, y_test

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter = 2000,verbose=1,n_jobs = -1)
LogisticRegression?

In [ ]:
model.fit(x_train,y_train)
print(accuracy(model.predict(x_train),y_train))
print(accuracy(model.predict(x_test),y_test))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.8105939004815409
0.7985074626865671


In [ ]:
# sklearn logistic regression
# training accuracy 80.41%
# cv accuracy 78.35% .....performs similar to my model haha!

In [ ]:
pred = model.predict(scaled_leaderboard_test)

In [ ]:
result4 = pd.DataFrame({"PassengerId":Test_ID,"Survived":pred})


In [ ]:
result4

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
model.coef_

array([[-0.77535817, -1.28456945, -0.433674  , -0.10679521,  0.17599029,
        -0.31982249, -0.0325896 ,  0.02908175, -0.22899953]])

In [ ]:
model.intercept_

array([-0.64511945])

In [ ]:
# for comparison my model outputted
#  w = array([[-0.73407141, -1.33721458, -0.42440414, -0.22498804,  0.27944166, -0.35686605,  0.01487571,  0.00288802, -0.04944191]])
# b = -0.6780945346813919

In [ ]:
result4.to_csv("sklearn_logistic_reg.csv",index=False, header = True)  #errors :(, also i'll use the feature engineered data soon!


In [ ]:
# currently i'm not able to get the result4 csv due to the error above so now i'll fit the model on the feature
# engineered data

In [ ]:
# so  training accuracy 80.41%
#     cv accuracy 78.35%
#     leaderboard accuracy 77.272%

In [ ]:
data

,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,FareBin_Mid,FareBin_High,...,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Pclass_2,Pclass_3
0,1,22,720,7,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,38,816,71,1,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,0,26,914,7,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,35,65,53,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4,1,35,649,8,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1,29,711,8,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
414,0,39,834,108,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,1,38,872,7,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
416,1,29,579,8,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
# we have data which we need to separate train-test set (i know many of you'll think about data leakage which i'm aware of but don't understand yet)
# and i combined the train-test set for faster preprocessing and feature engineering


In [59]:
train_engineered = data.iloc[:891].copy()
train_engineered

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Title,FareBin
0,1,3,1,22.000000,720,7.2500,1,0,0,1,12,1
1,2,1,0,38.000000,816,71.2833,1,1,0,0,13,0
2,3,3,0,26.000000,914,7.9250,0,0,0,1,9,1
3,4,1,0,35.000000,65,53.1000,1,0,0,1,13,0
4,5,3,1,35.000000,649,8.0500,0,0,0,1,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,1,27.000000,130,13.0000,0,0,0,1,15,1
887,888,1,0,19.000000,18,30.0000,0,0,0,1,9,2
888,889,3,0,29.881138,923,23.4500,3,0,0,1,9,2
889,890,1,1,26.000000,11,30.0000,0,1,0,0,12,2


In [60]:
test_engineered = data.iloc[891:].copy()
test_engineered

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Family,Embarked_C,Embarked_Q,Embarked_S,Title,FareBin
0,892,3,1,34.500000,376,7.8292,0,0,1,0,12,1
1,893,3,0,47.000000,582,7.0000,1,0,0,1,13,1
2,894,2,1,62.000000,184,9.6875,0,0,1,0,12,1
3,895,3,1,27.000000,366,8.6625,0,0,0,1,12,1
4,896,3,0,22.000000,338,12.2875,2,0,0,1,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.881138,711,8.0500,0,0,0,1,12,1
414,1306,1,0,39.000000,834,108.9000,0,1,0,0,3,3
415,1307,3,1,38.500000,872,7.2500,0,0,0,1,12,1
416,1308,3,1,29.881138,579,8.0500,0,0,0,1,12,1


In [72]:
train_engineered_scaled = scaler.fit_transform(train_engineered)
test_engineered_scaled = scaler.transform(test_engineered)

In [62]:
x_engineered_train, x_engineered_test, y_train_ ,y_test_ = splitter(train_engineered_scaled,y,test_size = 0.3)

In [63]:
x_engineered_train.shape

(623, 12)

In [70]:
# predicting on feature engineered label encoded dataset this time, below this will be models on one hot encoded literally messed up 33 columns data
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(max_iter = 2000)
model_log.fit(x_engineered_train,y_train_)
accuracy(y_test_, model_log.predict(x_engineered_test))  #nicee, cv 76.492

0.7649253731343284

In [71]:
accuracy(y_train_, model_log.predict(x_engineered_train)) #train 82.02 , doesnt seem like a big improvement to me as compared to predicted on normal preprocessed data

0.8202247191011236

In [76]:
prein = model_log.predict(test_engineered_scaled)
resul = pd.DataFrame({"PassengerId":Test_ID,"Survived":prein})
resul.to_csv("Log reg on label encoded feature engineered2.csv",index=False,header=True) #75.358%
# now left to do.  use the label encoded feature eng non scaled data to predict using boosted trees

In [75]:
resul

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [77]:
xgb_engineered_train, xgb_engineered_test, ygb_train_ ,ygb_test_ = splitter(train_engineered,y,test_size = 0.3)

In [79]:
m_cat.fit(xgb_engineered_train,ygb_train_)
accuracy(ygb_test_,m_cat.predict(xgb_engineered_test)) #79.477

Learning rate set to 0.008417
0:	learn: 0.6864116	total: 2.51ms	remaining: 2.51s
1:	learn: 0.6803832	total: 4.83ms	remaining: 2.41s
2:	learn: 0.6761155	total: 5.77ms	remaining: 1.92s
3:	learn: 0.6708595	total: 7.29ms	remaining: 1.81s
4:	learn: 0.6658186	total: 8.35ms	remaining: 1.66s
5:	learn: 0.6606345	total: 9.44ms	remaining: 1.56s
6:	learn: 0.6553248	total: 12ms	remaining: 1.71s
7:	learn: 0.6497246	total: 13.5ms	remaining: 1.67s
8:	learn: 0.6445231	total: 14.9ms	remaining: 1.64s
9:	learn: 0.6390752	total: 17.3ms	remaining: 1.72s
10:	learn: 0.6348669	total: 19.5ms	remaining: 1.75s
11:	learn: 0.6297054	total: 21.7ms	remaining: 1.79s
12:	learn: 0.6247767	total: 28.1ms	remaining: 2.13s
13:	learn: 0.6197665	total: 30.3ms	remaining: 2.13s
14:	learn: 0.6157233	total: 37.6ms	remaining: 2.47s
15:	learn: 0.6122135	total: 40.9ms	remaining: 2.52s
16:	learn: 0.6072174	total: 42.5ms	remaining: 2.46s
17:	learn: 0.6025502	total: 44.4ms	remaining: 2.42s
18:	learn: 0.5981329	total: 48.2ms	remaining: 

0.7947761194029851

In [80]:
accuracy(ygb_train_,m_cat.predict(xgb_engineered_train))  #94.454

0.9454253611556982

In [82]:
precat = m_cat.predict(test_engineered)
resultcat = pd.DataFrame({"PassengerId":Test_ID,"Survived":precat})
resultcat.to_csv("catboost on label encoded feature engineered.csv",index=False,header=True) #77.751

In [83]:
# i think im messing up with datasets a lot and scaling, preprocessing, feature engineering, adding models over time messes up a lot,plus code snippet issues in prev
# models accuracy make me skeptical a bit as well and maybe data leakage as well, and variable names....
# left to do basically make a new less messy and more structured, bagging ensemble of logistic reg, best model here to perform on leaderbaord is
# catboost on label encoded data - 77.751 accuracy,thats better then my best ever kernel of RF at 78.708 (not rally, im tripping) , and also better then catboost on basic
# preprocessed data which is at 76.076

In [ ]:
model2 = LogisticRegression(max_iter = 2000)
model2.fit(x_engineered_train,y_train)
accuracy(y_test, model2.predict(x_engineered_test))

0.6119402985074627

In [ ]:
accuracy(y_train, model2.predict(x_engineered_train))

0.6211878009630819

In [ ]:
# training accuracy is 82.825%
# cv accuracy is 82.835% , looks a bit fishy,but promising as well, i'll rerun quickly rerun the cells, and maybe this is what data
# leakage causes?
# still same after rerunning, maybe i'll change the hyperparameters
# turns out initailly i put predict before the y label in accuracy score, leading to higher scores, actual scores - training = 62.11, cv = 61.19, leaderboard = 62.44

In [ ]:
model3 = LogisticRegression(max_iter = 2500) #max_iter changed to 2500
model3.fit(x_engineered_train,y_train)
print(accuracy(y_test, model3.predict(x_engineered_test)))
accuracy(y_train, model3.predict(x_engineered_train))  # accuracy still exact same, i'll try cross val score and also predict on leaderbaord data

0.6119402985074627


0.6211878009630819

In [ ]:
predn = model2.predict(test_engineered_scaled)

In [ ]:
predn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
result5 = pd.DataFrame({"PassengerId":Test_ID,"Survived":predn})
result5

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [ ]:
result5 = pd.DataFrame({"PassengerId":Test_ID,"Survived":pred})
result5["Survived"].value_counts()

Survived
0    263
1    155
Name: count, dtype: int64

In [ ]:
result5.to_csv("feature_engineered.csv",index=False, header = True)